In [291]:
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torchvision.models as models
from sklearn.metrics import classification_report
from torch.nn import functional as F
from torch import nn

In [292]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Chuẩn bị dữ liệu

In [293]:
#hàm biến đổi dữ liệu 
def prepare_data(trainset, testset):
  transform_train = transforms.Compose([
                                  transforms.Resize((256, 256)),                                         #resize images
                                  transforms.RandomCrop(224, padding=4),                 #lấy vùng ngẫu nhiên trong ảnh
                                  transforms.RandomRotation(degrees= 20),
                                  transforms.RandomHorizontalFlip(),                             #lat ngang
                                  transforms.ToTensor(),                                                       #convert to Tensor
                                  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])

  transform_test = transforms.Compose([
                                       transforms.Resize((224, 224)),
                                       transforms.ToTensor(),                           
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])
  
  path_data_train = "/content/drive/MyDrive/ki_1_4/Xu_ly_anh/Thay_cuong/giua_ki/data/train"
  path_data_test = "/content/drive/MyDrive/ki_1_4/Xu_ly_anh/Thay_cuong/giua_ki/data/test"

  trainset = torchvision.datasets.ImageFolder(root=path_data_train, transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root=path_data_test, transform=transform_test)

  return trainset, testset

In [294]:
#hàm load dữ liệu thành các batch để xử lý 
def prepare_loader(trainset, testset):
  batch_size = 8
  trainloader = torch.utils.data.DataLoader(
      trainset, batch_size=batch_size, shuffle=True, num_workers=2)
  testloader = torch.utils.data.DataLoader(
      testset, batch_size=batch_size, shuffle=False, num_workers=2)
  return trainloader, testloader

#Mô hình

In [295]:
class MyNet(nn.Module):
  def __init__(self):
    super().__init__()          #gọi hàm khởi tạo của lớp nn để bắt đầu một modun 
    self.features = models.resnet18(pretrained=True)
    self.features.fc = torch.nn.Linear(512, 4)        #chuyển thành 4 classes 
  
  def forward(self, x):
    out = self.features(x)
    return out

#Train

In [296]:
def training_step(epoch, model, train_loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0  #gía trị hàm loss 
  reporting_step = 8

  for i, (images, labels) in enumerate(train_loader):
    images, labels = images.to(device), labels.to(device)

    #không tích luỹ đạo hàm 
    optimizer.zero_grad()
    outputs = model(images)                 
    loss = loss_func(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % reporting_step == reporting_step - 1:
      print(f"Epoch: {epoch} step: {i} average loss {running_loss / reporting_step:0.4f}")
      running_loss = 0.0

def test_step(epoch, model, test_loader, device):
  model.eval()
  y_pred = []
  y_true = []
  
  for i, (images, labels) in enumerate(test_loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs, dim=1)
    y_pred += list(predicted.cpu().numpy())
    y_true += list(labels.cpu().numpy())
  return y_pred, y_true

#loss và optimizer 
def get_trainer(model):
  loss = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  return loss, optimizer

#Hàm main

In [297]:
def main(PATH='/content/drive/MyDrive/ki_1_4/Xu_ly_anh/Thay_cuong/giua_ki/model_simpleOCR10.pth'):
  classes = ['highlands', 'others', 'phuclong', 'starbucks']
  #datasets = prepare_data()         #convert dữ liệu
  trainset = testset = trainloaders = testloaders = transforms.ToTensor()

  datasets_train, datasets_test = prepare_data(trainset, testset)

  print("data", len(datasets_train), len(datasets_test))

  train_loaders, test_loaders = prepare_loader(datasets_train, datasets_test)                         #đọc dữ liệu vào từng batch một 
  model_simpleOCR = MyNet()

  #lưu lại model train 
  model_simpleOCR.load_state_dict(torch.load('/content/drive/MyDrive/ki_1_4/Xu_ly_anh/Thay_cuong/giua_ki/model_simpleOCR10.pth'))

  loss, optimizer = get_trainer(model_simpleOCR)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  number_epoch = 11

  print("number class: ", len(classes))

  #chuyển mô hình huấn luyện lên device 
  model_simpleOCR.to(device)

  for epoch in range(number_epoch):
    print('----------------------------------------------------------')
    training_step(epoch, model_simpleOCR, train_loaders, loss, optimizer, device)
    y_pred, y_true = test_step(epoch, model_simpleOCR, test_loaders, device)

    print(classification_report(y_true, y_pred, target_names=classes))

    torch.save(model_simpleOCR.state_dict(), PATH)

  
  return model_simpleOCR

model_simpleOCR = main()

data 495 100
number class:  4
----------------------------------------------------------
Epoch: 0 step: 7 average loss 0.3606
Epoch: 0 step: 15 average loss 0.2176
Epoch: 0 step: 23 average loss 0.1318
Epoch: 0 step: 31 average loss 0.1877
Epoch: 0 step: 39 average loss 0.0931
Epoch: 0 step: 47 average loss 0.1386
Epoch: 0 step: 55 average loss 0.1381
              precision    recall  f1-score   support

   highlands       0.95      0.97      0.96        40
      others       1.00      1.00      1.00        18
    phuclong       1.00      0.88      0.94        26
   starbucks       0.83      0.94      0.88        16

    accuracy                           0.95       100
   macro avg       0.95      0.95      0.95       100
weighted avg       0.95      0.95      0.95       100

----------------------------------------------------------
Epoch: 1 step: 7 average loss 0.1292
Epoch: 1 step: 15 average loss 0.4215
Epoch: 1 step: 23 average loss 0.4634
Epoch: 1 step: 31 average loss 0.1744
E